In [2]:
import pandas as pd
import numpy as np
import platform

In [6]:
if platform.system() == 'Windows':
    path = '.\data\STOCK.csv'
else:
    path = './data/STOCK.csv'

In [7]:
df = pd.read_csv(path)

In [8]:
# How to define anomaly truth label

# Let define it as stock that suddenly rise in one day for over 3-5 times, but suddenly drop afterward
# then it is an anomaly stock

,Unnamed: 0,259 HK Equity,69 HK Equity,374 HK Equity,285 HK Equity,270 HK Equity,102 HK Equity,871 HK Equity,533 HK Equity,629 HK Equity,...,8573 HK Equity,9995 HK Equity,1351 HK Equity,2960 HK Equity,2959 HK Equity,15591 HK Equity,2955 HK Equity,2956 HK Equity,16567 HK Equity,2957 HK Equity
0,1/9/2020,1.63,6.45,2.760,33.55,11.92,0.540,0.053,1.50,0.285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2/9/2020,1.63,6.60,2.760,38.25,11.90,0.540,0.057,1.49,0.285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3/9/2020,1.62,6.63,2.695,39.00,12.02,0.540,0.054,1.47,0.280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4/9/2020,1.63,6.49,2.690,41.35,12.10,0.540,0.056,1.46,0.280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7/9/2020,1.63,6.39,2.710,38.30,12.18,0.520,0.055,1.48,0.280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,8/9/2020,1.59,6.36,2.700,36.85,12.68,0.540,0.055,1.48,0.295,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,9/9/2020,1.62,6.38,2.700,37.30,12.68,0.540,0.056,1.48,0.280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10/9/2020,1.63,6.54,2.690,37.20,12.74,0.540,0.054,1.47,0.280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,11/9/2020,1.63,6.73,2.690,37.90,12.88,0.540,0.052,1.49,0.280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,14/9/2020,1.64,6.62,2.690,40.00,12.86,0.540,0.054,1.49,0.280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
